Declaration with PyTorch's Dataset heritance


In [6]:
import pandas as pd
import torch
import glob
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, TensorDataset
from music21 import instrument as m21instrument
from time import sleep

def performanceDecode(performance: pd.DataFrame):
        '''
            Receive a EPMS Performance Block and separe
            the data into a Multi Hot Encoding (MHE) representation
            of the notes played and a int (vector?) with
            the velocity of the notes (range 0-127)
        '''
        performance = performance.to_numpy()
        # print(f'perf Shape: {performance.shape}')
        if performance.shape[0] == 0:
            return None

        mhe = np.zeros(performance.shape).astype(int)
        # print(mhe.shape)
        velocities = np.zeros(performance.shape).astype(float)

        for i, frame in enumerate(performance):
            # print(frame)
            for j, note_value in enumerate(frame):
                if type(note_value) is bool:
                    mhe[i][j] = 0
                    velocities[i][j] = 0
                else:
                    mhe[i][j] = 1
                    velocities[i][j] = note_value

        return [mhe, velocities]


class InstrumentDataset(Dataset):
    '''
        pianoDataset = InstrumentDataset(path, m21.instruments.Piano)
        pianoDataLoader = DataLoader(pianoDataset, batch_size=1, shuffle=True)

        for song in pianoDataLoader:
            pass
    '''

    def __init__(self, root_dir:str, instrument:str, keyboard_size:int):
        self.root_dir = root_dir
        self.instrument = instrument
        self.keyboard_size = keyboard_size
        self.dataset_files = glob.glob(f'{root_dir}**/**.pkl', recursive=True)


    def __len__(self):
        return len(self.dataset_files)


    def __getitem__(self, idx):
        # retorna um MHE e um VELOCITIES
        pkl_file_path = Path(self.dataset_files[idx])
        # print(pkl_file_path)
        file_data = pd.read_pickle(pkl_file_path)
        # print(file_data)
        # print(type(self.instrument))
        instrument_data = file_data[file_data.INSTRUMENT==self.instrument]
        # print(instrument_data)
        # print

        # print(len(instrument_data))
        # if len(instrument_data == 0):
        #     return [1]

        # print(instrument_data)
        performance = instrument_data.iloc[:, (len(instrument_data.columns) - self.keyboard_size):]

        result = performanceDecode(performance)

        if result is None:
            return []

        return result

class TrainingSamples(Dataset):
    '''
        training_samples = TrainingSamples(song, 16, 88)

        # DL depth = 1
        training_dataloader = DataLoader(training_samples, batch_size=1, shuffle=False)

        # DL depth = 4
        training_dataloader = DataLoader(training_samples, batch_size=4, shuffle=False)

        for sample in training_dataloader:
            agent.predictor.train()
    '''

    def __init__(self, samples:np.array, resolution:int):
        # recebe um Dataframe
        self.samples = samples
        # print(f'[TrainingSamples Shape] {self.samples.shape}')
        self.resolution = resolution

    def __len__(self):
        return len(self.samples) - self.resolution - 1

    def __getitem__(self, idx):
        # retorna um dataframe do tamannho da resolucao
        sample_no_shift = np.array(self.samples[idx:idx + self.resolution, :])
        sample_shiftted = np.array(self.samples[idx+1:idx + self.resolution+1, :])
        # only returning mhe for now
        return (torch.from_numpy(sample_no_shift).float(), torch.from_numpy(sample_shiftted).float())